In [48]:
!pip install mysql-connector-python

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  # es para enseñar todas las columnas del df
pd.set_option('display.max_rows', None)     # es para enseñar todas las filas del df


import warnings
warnings.simplefilter('ignore')   # es para quitar warnings

In [49]:
import mysql.connector as conn

## ACTOR

In [2]:
df_actor=pd.read_csv('../data/actor.csv')


In [3]:
# Elimino la columna 'last_update'

df_actor = df_actor.drop(['last_update'], axis=1)

In [4]:
df_actor.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   actor_id    200 non-null    int64 
 1   first_name  200 non-null    object
 2   last_name   200 non-null    object
dtypes: int64(1), object(2)
memory usage: 26.2 KB


In [5]:
len(df_actor.drop_duplicates())==len(df_actor)  # no hay duplicados

True

## CATEGORY

In [6]:
df_category=pd.read_csv('../data/category.csv')

In [7]:
# Elimino la columna 'last_update'

df_category = df_category.drop(['last_update'], axis=1)

In [8]:
# Crear una fila nueva con una nueva categoría.

df2 = pd.DataFrame({'category_id': [99],
                    'name' : ['Unknown']})

df_category = pd.concat([df_category, df2], ignore_index = True, axis = 0)

## FILM

In [9]:
df_film=pd.read_csv('../data/film.csv')

## INVENTORY

In [10]:
df_inventory=pd.read_csv('../data/inventory.csv')

In [11]:
# Elimino la columna 'last_update'

df_inventory = df_inventory.drop(['last_update'], axis=1)

## LANGUAGE

In [12]:
df_language=pd.read_csv('../data/language.csv')


In [13]:
# Elimino la columna 'last_update'

df_language = df_language.drop(['last_update'], axis=1)

## RENTAL

In [14]:
df_rental=pd.read_csv('../data/rental.csv')

In [15]:
# Cambio el 'return_date' a formato 'date'

df_rental.rental_date = pd.to_datetime(df_rental.rental_date, format= '%Y-%m-%d')
df_rental.return_date = pd.to_datetime(df_rental.return_date, format= '%Y-%m-%d')

In [16]:
# Elimino la columna 'last_update'

df_rental = df_rental.drop(['last_update'], axis=1)

### OLD_HDD - Relaciones titulo, actor y categoría

In [17]:
df_old_HDD=pd.read_csv('../data/old_HDD.csv')

In [18]:
df_old_HDD.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   first_name    1000 non-null   object
 1   last_name     1000 non-null   object
 2   title         1000 non-null   object
 3   release_year  1000 non-null   int64 
 4   category_id   1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 207.6 KB


# Modificar y limpiar

## Modifico tabla film

In [19]:
# Hago un left join en 'df_film' para obtener 'category_id', haciendo que en films y old_HDD el index sea title.


film = df_film.set_index('title').join(df_old_HDD[["title", "category_id"]].set_index('title'), lsuffix='', rsuffix='_dcha', how='left')


In [20]:
# Le elimino duplicado que se generan

film.drop_duplicates(inplace=True)

In [21]:
# Le reseteo el indice

film.reset_index(inplace=True)

In [22]:
# Relleno en la tabla film la columna category_id con 99 

film.category_id.fillna(99, inplace=True)

# Modificar la columna de category_id

film.category_id = film.category_id.astype(dtype='int64')

In [23]:
# Elimino la columna de original_language_id

film.drop('original_language_id', axis=1, inplace=True)

In [24]:
# Elimino la columna 'last_update'

film = film.drop(['last_update'], axis=1)

## Creo tabla actor_film

In [25]:
actor = df_actor.copy(deep=True)

In [26]:
# Creo una columna y concateno first name y last name

actor['name'] = actor['first_name'] + ' ' + actor['last_name']

In [27]:
hdd = df_old_HDD

In [28]:
# Creo una columna nueva y concateno first name y last name

hdd['name'] = hdd['first_name'] + ' ' + hdd['last_name']

In [29]:
# Creo la tabla 'actor_film' que voy a utilizar como relación many to many entre actor y film.

# Primero uso hdd para hacer un join a través del indice 'title' y obtener 'film_id'

actor_film = hdd.set_index('title').join(film[["title", "film_id"]].set_index('title'), lsuffix='', rsuffix='_dcha', how='left')


In [30]:
# Elimino duplicados por si se generaran

actor_film.drop_duplicates(inplace=True)

In [31]:
# Reseteo el indice para que la columna 'title' vuelva a ser una columna

actor_film.reset_index(inplace=True)

In [32]:
# Primero uso 'actor_film' para hacer un join a través del indice 'name' y obtener 'actor_id'

actor_film = actor_film.set_index('name').join(actor[["name", "actor_id"]].set_index('name'), lsuffix='', rsuffix='_dcha', how='left')

In [33]:
# Elimino duplicados y reseteo el índice

actor_film.drop_duplicates(inplace=True)
actor_film.reset_index(inplace=True)

In [34]:
# Sobreescribo 'actor_film' solo con las columnas de 'actor_id' y 'film_id'

actor_film = actor_film[['actor_id', 'film_id']]

## Compruebo que todas las tablas este perfectas para subir

In [47]:
df_rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1


In [46]:
df_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   inventory_id  1000 non-null   int64
 1   film_id       1000 non-null   int64
 2   store_id      1000 non-null   int64
dtypes: int64(3)
memory usage: 23.6 KB


In [54]:
password = "".join(open('../text/mysql_pass.txt', "r").readlines())


'Bvmovement1$'

In [57]:
# Conecta a una base de datos mysql y activa el cursor/motor

password = "".join(open('../text/mysql_pass.txt', "r").readlines()) # Me traigo el password de mysql

crea_db=conn.connect(host='localhost', user='root', passwd=password)

cursor=crea_db.cursor()



In [58]:
# Crear base de datos

cursor.execute('drop database if exists blockbuster;')
cursor.execute('create database blockbuster;')

In [59]:
# conecta a una base de datos

db=conn.connect(host='localhost', user='root', passwd=password, database='blockbuster')

cursor=db.cursor()

In [60]:
# check

cursor.execute('show databases;')

for x in cursor:
    print(x)

('blockbuster',)
('information_schema',)
('mydb',)
('mysql',)
('performance_schema',)
('publications',)
('sys',)


In [ ]:
# crea tabla


# cursor.execute('drop table if exists zapatos;')



tabla='''

        create table zapatos (
            
            id int primary key,
            brand varchar(50),
            color varchar(50),
            size varchar(50),
            price float,
            url varchar(10000),
            ean varchar(50)
        );

'''


# cursor.execute(tabla)


